In [1]:
import create_model
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import os
from tec_reduced_model.set_parameters import (
    set_thermal_parameters,
    set_experiment_parameters,
    set_ambient_temperature,
)

In [2]:
# Change simulation parameters here
temperature = 25  # in degC, valid values: 0, 10, 25
crate = 1         # valid values: 0.5, 1, 2
#cells_ignore = ["790","791","792"]
cell = ["789"]   # cells to ignore, in this case we ignore cell 791 because the data for that cell is faulty

In [3]:
model=create_model.model_build(temperature,crate,cell)


In [4]:
%time
from scipy import optimize
x0 =np.array([5e-14,6.48e-7,20,2.85e3])
minimum = optimize.fmin(model.fitness, x0, xtol=1e-3, ftol=1e-3)

CPU times: total: 0 ns
Wall time: 0 ns


psetup failed: D:\bld\casadi_1636131388751\work\casadi\interfaces\sundials\cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: D:\bld\casadi_1636131388751\work\casadi\interfaces\sundials\cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: D:\bld\casadi_1636131388751\work\casadi\interfaces\sundials\cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: D:\bld\casadi_1636131388751\work\casadi\interfaces\sundials\cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: D:\bld\casadi_1636131388751\work\casadi\interfaces\sundials\cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: D:\bld\casadi_1636131388751\work\casadi\interfaces\sundials\cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: D:\bld\casadi_1636131388751\work\casadi\interfaces\sundials\cvodes_interface.cpp:650: 'jacF' calculation failed
psetup failed: D:\bld\casadi_1636131388751\work\casadi\interfaces\sundials\cvodes_interface.cpp:650: 'jacF' cal

SolverError: Maximum number of decreased steps occurred at t=40.0. Try solving the model up to this time only or reducing dt_max (currently, dt_max=0.4432270615486007).

In [ ]:
# Define the TSPMe model
model = pybamm.lithium_ion.SPMe(
    options={
        "thermal": "lumped",
        "dimensionality": 0,
        "cell geometry": "arbitrary",
        "electrolyte conductivity": "integrated",
    },
    name="TSPMe",
)

#param["Negative electrode diffusivity [m2.s-1]"] = pos[0]
#param.update({
#        "Negative electrode exchange-current density [A.m-2]": j0_neg,
#        "Negative electrode reaction coefficient": "[input]",
#    }, check_already_exists=False)

# Define the experiment to be a CC discharge at the defined C-rate followed by a 2-hour relaxation
experiment = pybamm.Experiment(
    [
        "Discharge at {}C until 2.5 V (5 seconds period)".format(Crate),
        "Rest for 2 hours",
    ],
    period="30 seconds",
)

# Solve the model
simulation = pybamm.Simulation(
    model,
    parameter_values=param,
    experiment=experiment,
)
simulation.solve( inputs={"Negative electrode diffusivity [m2.s-1]":minimum[0],
                          "Negative electrode reaction coefficient": minimum[1],
                          "Total heat transfer coefficient [W.m-2.K-1]":minimum[2],
                          "Positive current collector specific heat capacity [J.kg-1.K-1]":minimum[3],
                          "Negative current collector specific heat capacity [J.kg-1.K-1]":minimum[3],
                          "Negative electrode specific heat capacity [J.kg-1.K-1]":minimum[3],
                          "Separator specific heat capacity [J.kg-1.K-1]":minimum[3],
                          "Positive electrode specific heat capacity [J.kg-1.K-1]":minimum[3]
                         })
solution = simulation.solution


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(11, 4.5))
dataset = import_thermal_data(Crate, temperature)
data_conc = {"time": [], "voltage": [], "temperature": []}

for cell, data in dataset.items():
    if cell in cells_ignore:
        continue

    idx_start, idx_end = get_idxs(data, Crate * 5, 5 / 3)
    if len(idx_end) == 1:
        idx_end = np.append(idx_end, len(data["Time [s]"]))

    axes[0].plot(
        data["Time [s]"][idx_start[0] : idx_end[1]]
        - data["Time [s]"][idx_start[0]],
        data["Voltage [V]"][idx_start[0] : idx_end[1]],
        label=cell,
    )
    axes[1].plot(
        data["Time [s]"][idx_start[0] : idx_end[1]]
        - data["Time [s]"][idx_start[0]],
        data["Temp Cell [degC]"][idx_start[0] : idx_end[1]],
        label=cell,
    )

    data_conc["time"] = np.append(
        data_conc["time"],
        data["Time [s]"][idx_start[0] : idx_end[1]]
        - data["Time [s]"][idx_start[0]],
    )
    data_conc["voltage"] = np.append(
        data_conc["voltage"], data["Voltage [V]"][idx_start[0] : idx_end[1]]
    )
    data_conc["temperature"] = np.append(
        data_conc["temperature"],
        data["Temp Cell [degC]"][idx_start[0] : idx_end[1]],
    )

    
axes[0].plot(
    solution["Time [s]"].entries,
    solution["Terminal voltage [V]"].entries,
    color="black",
    label="TSPMe",
)

axes[0].scatter(
    0,
    solution["X-averaged battery open circuit voltage [V]"].entries[0],
    marker="x",
    color="black",
)

axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel("Voltage (V)")

axes[1].plot(
    solution["Time [s]"].entries,
    solution["X-averaged cell temperature [K]"].entries - 273.15,
    color="black",
    label="TSPMe",
)

axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Cell temperature (°C)")
axes[1].legend()
    
fig.suptitle("Ambient temperature: {} °C, C-rate: {}C".format(temperature, Crate))    

fig.tight_layout()
fig.subplots_adjust(top=0.9)